In [ ]:
# Importing required librarie
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# This code we have taken the file as .csv and uploaded it
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
# This code reads two CSV files, 'tmdb_5000_movies.csv' and 'tmdb_5000_credits.csv', and stores the data in two DataFrames named 'movies' and 'credits', respectively.
movies.head(2)

In [ ]:
# This displays the first two rows of the 'movies' DataFrame, providing a glimpse of its contents.
movies.shape

In [ ]:
# This shows the dimensions (number of rows and columns) of the 'movies' DataFrame.
credits.head()

In [ ]:
# This displays the first few rows of the 'credits' DataFrame, showing some of its contents.
movies = movies.merge(credits,on='title')

In [ ]:
# This merges the 'movies' and 'credits' DataFrames based on the 'title' column, combining their data into a new 'movies' DataFrame.
movies.head()

In [ ]:
# This displays the first few rows of the 'movies' DataFrame after the merge operation.
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
# This creates a new 'movies' DataFrame containing only specific columns, including 'movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', and 'crew'.
movies.head()

In [ ]:
# This displays the first few rows of the 'movies' DataFrame after selecting specific columns.
import ast

In [ ]:
# This imports the 'ast' module, which provides functions to handle Abstract Syntax Trees (AST) in Python.
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
# This defines a function 'convert' that takes a text input, evaluates it as a Python data structure (a list of dictionaries), and extracts the 'name' key from each dictionary, returning a list of names.
movies.dropna(inplace=True)

In [ ]:
# This removes any rows with missing values (NaN) from the 'movies' DataFrame, updating it in place.
movies['genres'] = movies['genres'].apply(convert)
movies.head()

In [ ]:
# This applies the 'convert' function to each element of the 'genres' column in the 'movies' DataFrame. It extracts the 'name' key from the list of dictionaries and replaces the 'genres' column with the new list of names.
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
# This applies the 'convert' function to each element of the 'keywords' column in the 'movies' DataFrame. It extracts the 'name' key from the list of dictionaries and replaces the 'keywords' column with the new list of names.
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
# This is an example of using 'ast.literal_eval' to evaluate a string representation of a list of dictionaries containing genre information.
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [ ]:
# This defines a function 'convert3' that takes a text input, evaluates it as a Python data structure (a list of dictionaries), and extracts the 'name' key from the first three dictionaries, returning a list of names.
movies['cast'] = movies['cast'].apply(convert)
movies.head()

In [ ]:
# This applies the 'convert' function to each element of the 'cast' column in the 'movies' DataFrame. It extracts the 'name' key from the list of dictionaries and replaces the 'cast' column with the new list of names.
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [ ]:
# This applies a lambda function to each element of the 'cast' column in the 'movies' DataFrame. It keeps only the first three names in the list.
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
# This defines a function 'fetch_director' that takes a text input, evaluates it as a Python data structure (a list of dictionaries), and filters the dictionaries to find those with 'job' equal to 'Director', returning a list of director names.
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.sample(5)

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
new[new['title'] == 'The Lego Movie'].index[0]

In [ ]:
# This code displays the 'similarity' 2D array, showing the pairwise cosine similarity values between different movies.
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)



In [ ]:
# This code finds the index of the row in the 'new' DataFrame where the movie title is 'The Lego Movie'.
recommend('Gandhi')

In [ ]:
# This code defines a function 'recommend' that takes a movie title as input. It finds the index of the input movie in the 'new' DataFrame, then sorts the similarity scores between the input movie and all other movies in descending order. Finally, it prints the titles of the top 5 recommended movies based on similarity.
import pickle

In [ ]:
# This code calls the 'recommend' function with the movie title 'Gandhi', which will print the top 5 recommended movies similar to 'Gandhi'.
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
# This code saves the 'new' DataFrame and 'similarity' array into separate pickle files named 'movie_list.pkl' and 'similarity.pkl', respectively. Pickling is a way to store Python objects in a binary format for later use.
recommend("Batman Begins")